# Creating the feature table in BigQuery

## Configuration

In [194]:
from enum import Enum

In [195]:
number_of_addresses = 100

# format: dd.mm.yyyy
observation_period_start = "2020-02-01"
observation_period_end = "2020-02-01"

class ADDRESS_SELECTION(Enum):
    RANDOM = 1 # selects random addresses, that have been active within the observation period.
    RICHEST = 2 # selects the accounts that have the most ether # not yet implemented
    HIGHEST_TURNOVER = 3 # selects the accounts that have the most ether received + sent

address_selection = ADDRESS_SELECTION.RICHEST.value

# max USD amount to spent for executing sql queries
max_bigquery_costs_usd = 2

# Delete old tables
reset = True

## Creating Views

Setup

In [196]:
from google.cloud import bigquery
import simplejson as json
import os
from datetime import datetime
import pandas as pd

observation_period = (observation_period_start, observation_period_end) 
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "../secrets/bigquery-service-account.json"
client = bigquery.Client()
dataset_id = "ethereum_us"
table_names = []
table_ids = {}

Deleting all tables

In [197]:
table_ids_to_be_deleted = [t.table_id for t in client.list_tables(dataset_id)]

for table_id in table_ids_to_be_deleted:
    if reset == True or "view" in t.table_id or "tmp" in t.table_id:
        client.delete_table("{}.{}".format(dataset_id, table_id), not_found_ok=True)  

Create view "traces"

In [198]:
table_names.append("traces")
current_view_name = table_names[-1]
sql = """
      select * from `bigquery-public-data.crypto_ethereum.traces`
        where status = 1
            and DATE(block_timestamp) >= '{start}' 
            and DATE(block_timestamp) <= '{end}'
      """.format(start=observation_period[0], end=observation_period[1]); 

table_ids[current_view_name] = "{}_{}_{}_{}_{}_view".format(current_view_name, ADDRESS_SELECTION(address_selection).name, number_of_addresses, observation_period[0].replace("-","_"),observation_period[1].replace("-","_"))
view_ref = client.dataset(dataset_id).table(table_ids[current_view_name])
view = bigquery.Table(view_ref)
view.view_query = sql
view = client.create_table(view)  # API request
print("Successfully created view: '{}'".format(table_ids[current_view_name]))

Successfully created view: 'traces_RICHEST_100_2020_02_01_2020_02_01_view'


Create view "transactions"

In [199]:
table_names.append("transactions")
current_view_name = table_names[-1]
sql = """
      select * from `bigquery-public-data.crypto_ethereum.transactions`
        where receipt_status = 1
            and DATE(block_timestamp) >= '{start}' 
            and DATE(block_timestamp) <= '{end}'
      """.format(start=observation_period[0], end=observation_period[1]); 

table_ids[current_view_name] = "{}_{}_{}_{}_{}_view".format(current_view_name, ADDRESS_SELECTION(address_selection).name, number_of_addresses, observation_period[0].replace("-","_"),observation_period[1].replace("-","_"))
view_ref = client.dataset(dataset_id).table(table_ids[current_view_name])
view = bigquery.Table(view_ref)
view.view_query = sql
view = client.create_table(view)  # API request
print("Successfully created view: '{}'".format(table_ids[current_view_name]))

Successfully created view: 'transactions_RICHEST_100_2020_02_01_2020_02_01_view'


Create view "blocks"

In [200]:
table_names.append("blocks")
current_view_name = table_names[-1]
sql = """
      select * from `bigquery-public-data.crypto_ethereum.blocks`
        where 
            DATE(timestamp) >= '{start}' 
            and DATE(timestamp) <= '{end}'
      """.format(start=observation_period[0], end=observation_period[1]); 

table_ids[current_view_name] = "{}_{}_{}_{}_{}_view".format(current_view_name, ADDRESS_SELECTION(address_selection).name, number_of_addresses, observation_period[0].replace("-","_"),observation_period[1].replace("-","_"))
view_ref = client.dataset(dataset_id).table(table_ids[current_view_name])
view = bigquery.Table(view_ref)
view.view_query = sql
view = client.create_table(view)  # API request
print("Successfully created view: '{}'".format(table_ids[current_view_name]))

Successfully created view: 'blocks_RICHEST_100_2020_02_01_2020_02_01_view'


Create table "addresses" **(WARNING: this operation costs money!)**

In [201]:
table_names.append("addresses")
current_table_name = table_names[-1]
table_ids[current_table_name] = "{}_{}_{}_{}_{}_tmp".format(current_table_name, ADDRESS_SELECTION(address_selection).name, number_of_addresses, observation_period[0].replace("-","_"),observation_period[1].replace("-","_"))

job_config = bigquery.QueryJobConfig(destination="masterarbeit-245718.ethereum_us.{table_id}".format(table_id = table_ids["addresses"]))
job_config.dry_run = True
job_config.use_query_cache = False

if address_selection == ADDRESS_SELECTION.RANDOM.value:
    sql = """
                with addresses_traces as (
                    select from_address as address from masterarbeit-245718.ethereum_us.{table_id_traces}
                    UNION ALL
                    select to_address as address from masterarbeit-245718.ethereum_us.{table_id_traces}
                ), distinct_addresses as (
                    select distinct address from addresses_traces 
                )
                select address from distinct_addresses order by rand()
                LIMIT {number_of_addresses}
                """.format(table_id_traces=table_ids["traces"], number_of_addresses=number_of_addresses)
elif address_selection == ADDRESS_SELECTION.HIGHEST_TURNOVER.value:
    sql = """
                with double_entry_book as (
                    -- received wei
                    select to_address as address, value
                    from masterarbeit-245718.ethereum_us.{table_id_traces}
                    where to_address is not null
                    and status = 1
                    and (call_type not in ('delegatecall', 'callcode', 'staticcall') or call_type is null)
                    union all
                    -- sent wei
                    select from_address as address, value
                    from masterarbeit-245718.ethereum_us.{table_id_traces}
                    where from_address is not null
                    and status = 1
                    and (call_type not in ('delegatecall', 'callcode', 'staticcall') or call_type is null)
                )
                select address, sum(value) as turnover
                from double_entry_book
                group by address
                order by turnover desc
                limit {number_of_addresses}
                """.format(table_id_traces=table_ids["traces"], number_of_addresses=number_of_addresses)
elif address_selection == ADDRESS_SELECTION.RICHEST.value:
    sql = """
            with double_entry_book as (
                -- debits
                select to_address as address, value as value
                from masterarbeit-245718.ethereum_us.{table_id_traces}
                where to_address is not null
                and status = 1
                and (call_type not in ('delegatecall', 'callcode', 'staticcall') or call_type is null)
                union all
                -- credits
                select from_address as address, -value as value
                from masterarbeit-245718.ethereum_us.{table_id_traces}
                where from_address is not null
                and status = 1
                and (call_type not in ('delegatecall', 'callcode', 'staticcall') or call_type is null)
                union all
                -- transaction fees debits
                select miner as address, sum(cast(receipt_gas_used as numeric) * cast(gas_price as numeric)) as value
                from masterarbeit-245718.ethereum_us.{table_id_transactions} as transactions
                join masterarbeit-245718.ethereum_us.{table_id_blocks} as blocks on blocks.number = transactions.block_number
                group by blocks.miner
                union all
                -- transaction fees credits
                select from_address as address, -(cast(receipt_gas_used as numeric) * cast(gas_price as numeric)) as value
                from masterarbeit-245718.ethereum_us.{table_id_transactions}
            )
            select address, sum(value) as balance
            from double_entry_book
            group by address
            order by balance desc
            limit 40000
                """.format(table_id_traces=table_ids["traces"], number_of_addresses=number_of_addresses,
                          table_id_transactions=table_ids["transactions"],table_id_blocks=table_ids["blocks"])
    
query_job = client.query(sql, job_config=job_config)

cost_dollars = (query_job.total_bytes_processed / 10 ** 12) * 5

print("{} Megabytes will be processed".format(round(query_job.total_bytes_processed / 10 ** 6)))
print("It will cost ${}.".format(cost_dollars))

256 Megabytes will be processed
It will cost $0.00128230845.


In [202]:
job_config.dry_run = False

# Start the query, passing in the extra configuration.
if (cost_dollars > max_bigquery_costs_usd):
    raise Exception("Warning: This operation costs $ {}.".format(cost_dollars))

query_job = client.query(sql, job_config=job_config)  # Make an API request.
result = query_job.result()

max_bigquery_costs_usd -= cost_dollars
print("Successfully created table: '{}'".format(table_ids["addresses"]))

Successfully created table: 'addresses_RICHEST_100_2020_02_01_2020_02_01_tmp'


Create view "wei"

In [203]:
table_names.append("wei")
current_view_name = table_names[-1]
sql = """
            with weiView as (

              with wei_receivedView as (
                    
                  -- debits
                  select to_address, sum(ifnull(value, 0)) as wei_received
                  from masterarbeit-245718.ethereum_us.{table_id_traces}
                  where to_address is not null
                  and status = 1
                  and (call_type not in ('delegatecall', 'callcode', 'staticcall') or call_type is null) 
                  group by to_address
                    
              ), wei_sentView as (
              
                  -- credits
                  select from_address, sum(ifnull(value, 0)) as wei_sent
                  from  masterarbeit-245718.ethereum_us.{table_id_traces} 
                  where from_address is not null
                  and status = 1
                  and (call_type not in ('delegatecall', 'callcode', 'staticcall') or call_type is null) 
                  group by from_address
              ) 
              select 
                  CASE 
                    when to_address is not null then to_address
                    when from_address is not null then from_address
                  end as address, 
                  ifnull(wei_received,0) as wei_received, 
                  ifnull(wei_sent,0) as wei_sent
              from wei_receivedView full outer join wei_sentView on from_address = to_address
            ) 
            select address, wei_received, wei_sent from weiView right join masterarbeit-245718.ethereum_us.{table_id_addresses} using(address)
            """.format(table_id_traces=table_ids["traces"], table_id_addresses=table_ids["addresses"])

table_ids[current_view_name] = "{}_{}_{}_{}_{}_view".format(current_view_name, ADDRESS_SELECTION(address_selection).name, number_of_addresses, observation_period[0].replace("-","_"),observation_period[1].replace("-","_"))
view_ref = client.dataset(dataset_id).table(table_ids[current_view_name])
view = bigquery.Table(view_ref)
view.view_query = sql
view = client.create_table(view)  # API request
print("Successfully created view: '{}'".format(table_ids[current_view_name]))

Successfully created view: 'wei_RICHEST_100_2020_02_01_2020_02_01_view'


Create view "tx"

In [204]:
table_names.append("tx")
current_view_name = table_names[-1]
sql = """
            with txView as (

              with txSent as (
              
                  SELECT from_address, count(*) as number_of_tx_sent FROM masterarbeit-245718.ethereum_us.{table_id_traces}
                  where to_address is not null and status = 1 and (call_type not in ('delegatecall', 'callcode', 'staticcall') or call_type is null) 
                  group by from_address
                  
                ), txReceived as (
                
                  SELECT to_address, count(*) as number_of_tx_received FROM masterarbeit-245718.ethereum_us.{table_id_traces}
                  where to_address is not null and status = 1 and (call_type not in ('delegatecall', 'callcode', 'staticcall') or call_type is null) 
                  group by to_address
                ) 
                
                SELECT 
                    CASE  
                      WHEN to_address IS NOT NULL THEN to_address
                      WHEN from_address IS NOT NULL THEN from_address
                    END AS address,
                    IFNULL(number_of_tx_received, 0) as number_of_tx_received, 
                    IFNULL(number_of_tx_sent, 0) as number_of_tx_sent
                from txReceived FULL OUTER JOIN txSent on to_address = from_address
            ) 
            
            select address, number_of_tx_received, number_of_tx_sent from txView right join masterarbeit-245718.ethereum_us.{table_id_addresses} using(address)
            
            """.format(table_id_traces=table_ids["traces"], table_id_addresses=table_ids["addresses"])

table_ids[current_view_name] = "{}_{}_{}_{}_{}_view".format(current_view_name, ADDRESS_SELECTION(address_selection).name, number_of_addresses, observation_period[0].replace("-","_"),observation_period[1].replace("-","_"))
view_ref = client.dataset(dataset_id).table(table_ids[current_view_name])
view = bigquery.Table(view_ref)
view.view_query = sql
view = client.create_table(view)  # API request
print("Successfully created view: '{}'".format(table_ids[current_view_name]))

Successfully created view: 'tx_RICHEST_100_2020_02_01_2020_02_01_view'


Create view "avg_time_diff_received_tx"

In [205]:
table_names.append("avg_time_diff_received_tx")
current_view_name = table_names[-1]
sql = """
           with timeRecView as (

              with receivedTx as (
              
                SELECT to_address, count(*) as number_of_tx_received 
                FROM masterarbeit-245718.ethereum_us.{table_id_traces} 
                where to_address is not null 
                  and status = 1 
                  and (call_type not in ('delegatecall', 'callcode', 'staticcall') or call_type is null)
                group by to_address
              
              ), timeStampDiffs as (
              
                SELECT to_address, TIMESTAMP_DIFF(MAX(block_timestamp), MIN( block_timestamp ), second ) as timestampDiff
                FROM masterarbeit-245718.ethereum_us.{table_id_traces}
                where to_address is not null 
                  and status = 1 
                  and (call_type not in ('delegatecall', 'callcode', 'staticcall') or call_type is null)
                group by to_address
              
              ) 
              
              select to_address as address, 
                  CASE 
                    when (number_of_tx_received - 1)  > 0 then timestampDiff / (number_of_tx_received - 1) 
                    else 0
                  end as avgTimeDiffBetweenReceivedTransactions
              from receivedTx inner join  timeStampDiffs using(to_address)
            )
            
            select address, ifnull(avgTimeDiffBetweenReceivedTransactions,0) as avg_time_diff_received_tx from timeRecView right join masterarbeit-245718.ethereum_us.{table_id_addresses} using(address)         
            
            """.format(table_id_traces=table_ids["traces"], table_id_addresses=table_ids["addresses"])

table_ids[current_view_name] = "{}_{}_{}_{}_{}_view".format(current_view_name, ADDRESS_SELECTION(address_selection).name, number_of_addresses, observation_period[0].replace("-","_"),observation_period[1].replace("-","_"))
view_ref = client.dataset(dataset_id).table(table_ids[current_view_name])
view = bigquery.Table(view_ref)
view.view_query = sql
view = client.create_table(view)  # API request
print("Successfully created view: '{}'".format(table_ids[current_view_name]))

Successfully created view: 'avg_time_diff_received_tx_RICHEST_100_2020_02_01_2020_02_01_view'


Create view "avg_time_diff_sent_tx"

In [206]:
table_names.append("avg_time_diff_sent_tx")
current_view_name = table_names[-1]
sql = """
            with timeSentView as (

              with sentTx as (
              
                SELECT from_address, count(*) as number_of_tx_sent 
                FROM masterarbeit-245718.ethereum_us.{table_id_traces}
                where to_address is not null 
                    and status = 1 
                    and (call_type not in ('delegatecall', 'callcode', 'staticcall') or call_type is null)
                group by from_address
                
              ), timeStampDiffs as (
              
                SELECT from_address, TIMESTAMP_DIFF(MAX(block_timestamp), MIN( block_timestamp ), second ) as timestampDiff
                FROM masterarbeit-245718.ethereum_us.{table_id_traces}
                where to_address is not null 
                  and status = 1 
                  and (call_type not in ('delegatecall', 'callcode', 'staticcall') or call_type is null)
                group by from_address
                
            ) 
              
            select from_address as address, 
                CASE 
                    when (number_of_tx_sent - 1)  > 0 then 
                        timestampDiff / (number_of_tx_sent - 1) 
                    else 
                        0
                end as avgTimeDiffBetweenSentTransactions
            from sentTx inner join  timeStampDiffs using(from_address)
            )
            
            select address, ifnull(avgTimeDiffBetweenSentTransactions,0) as avg_time_diff_sent_tx from timeSentView right join masterarbeit-245718.ethereum_us.{table_id_addresses} using(address)        
            
            """.format(table_id_traces=table_ids["traces"], table_id_addresses=table_ids["addresses"])

table_ids[current_view_name] = "{}_{}_{}_{}_{}_view".format(current_view_name, ADDRESS_SELECTION(address_selection).name, number_of_addresses, observation_period[0].replace("-","_"),observation_period[1].replace("-","_"))
view_ref = client.dataset(dataset_id).table(table_ids[current_view_name])
view = bigquery.Table(view_ref)
view.view_query = sql
view = client.create_table(view)  # API request
print("Successfully created view: '{}'".format(table_ids[current_view_name]))

Successfully created view: 'avg_time_diff_sent_tx_RICHEST_100_2020_02_01_2020_02_01_view'


Create view "mined_blocks"

In [207]:
table_names.append("mined_blocks")
current_view_name = table_names[-1]
sql = """
            with mined_blocksView as (
                SELECT to_address as address, count(*) as mined_blocks 
                FROM masterarbeit-245718.ethereum_us.{table_id_traces}
                where trace_type = "reward"
                group by to_address
            )
            
            select address, ifnull(mined_blocks,0) as mined_blocks from mined_blocksView right join  masterarbeit-245718.ethereum_us.{table_id_addresses} using(address)

            """.format(table_id_traces=table_ids["traces"], table_id_addresses=table_ids["addresses"])

table_ids[current_view_name] = "{}_{}_{}_{}_{}_view".format(current_view_name, ADDRESS_SELECTION(address_selection).name, number_of_addresses, observation_period[0].replace("-","_"),observation_period[1].replace("-","_"))
view_ref = client.dataset(dataset_id).table(table_ids[current_view_name])
view = bigquery.Table(view_ref)
view.view_query = sql
view = client.create_table(view)  # API request
print("Successfully created view: '{}'".format(table_ids[current_view_name]))

Successfully created view: 'mined_blocks_RICHEST_100_2020_02_01_2020_02_01_view'


Create view "stddev_received_tx"

In [208]:
table_names.append("stddev_received_tx")
current_view_name = table_names[-1]
sql = """
            with timestamp_var as (
                
                with timestamps_diffs as (
                    
                    with timestamps_preceding_tx as (
                        
                        with timestamps_received_tx as (
                        
                            select to_address, block_timestamp 
                            from masterarbeit-245718.ethereum_us.{table_id_traces}
                            where to_address is not null 
                                and status = 1 
                                and (call_type not in ('delegatecall', 'callcode', 'staticcall') or call_type is null)
                        )
                        
                        select to_address, block_timestamp,
                            lag(block_timestamp) OVER (partition by to_address order by block_timestamp asc) as preceding_block_timestamp 
                        from timestamps_received_tx
                    )
                    
                    select to_address, block_timestamp, preceding_block_timestamp, 
                        TIMESTAMP_DIFF(block_timestamp, preceding_block_timestamp, second) as timestampdiff
                    from timestamps_preceding_tx
                )
                
                select to_address as address, STDDEV_SAMP(timestampdiff) as stddev_received_tx  
                from timestamps_diffs group by to_address 
            ) 
            
            select address, ifnull(stddev_received_tx, 0) as stddev_received_tx from timestamp_var right join masterarbeit-245718.ethereum_us.{table_id_addresses} using(address)
            
            """.format(table_id_traces=table_ids["traces"], table_id_addresses=table_ids["addresses"])

table_ids[current_view_name] = "{}_{}_{}_{}_{}_view".format(current_view_name, ADDRESS_SELECTION(address_selection).name, number_of_addresses, observation_period[0].replace("-","_"),observation_period[1].replace("-","_"))
view_ref = client.dataset(dataset_id).table(table_ids[current_view_name])
view = bigquery.Table(view_ref)
view.view_query = sql
view = client.create_table(view)  # API request
print("Successfully created view: '{}'".format(table_ids[current_view_name]))

Successfully created view: 'stddev_received_tx_RICHEST_100_2020_02_01_2020_02_01_view'


Create view "stddev_sent_tx"

In [209]:
table_names.append("stddev_sent_tx")
current_view_name = table_names[-1]
sql = """
            with timestamp_var as (
            
                with timestamps_diffs as (
                    
                    with timestamps_preceding_tx as (
                        
                        with timestamps_sent_tx as (
                        
                            select from_address, block_timestamp 
                            from masterarbeit-245718.ethereum_us.{table_id_traces}
                            where from_address is not null 
                                and status = 1 
                                and (call_type not in ('delegatecall', 'callcode', 'staticcall') or call_type is null)
                        )
                        
                        select from_address, block_timestamp,
                            lag(block_timestamp) OVER (partition by from_address order by block_timestamp asc) as preceding_block_timestamp 
                        from timestamps_sent_tx
                    )
                    
                    select from_address, block_timestamp, preceding_block_timestamp, 
                        TIMESTAMP_DIFF(block_timestamp, preceding_block_timestamp, second) as timestampdiff
                    from timestamps_preceding_tx
                )
                
                select from_address as address, STDDEV_SAMP(timestampdiff) as stddev_sent_tx  
                from timestamps_diffs group by from_address 
            ) 
            
            select address, ifnull(stddev_sent_tx,0) as stddev_sent_tx from timestamp_var right join masterarbeit-245718.ethereum_us.{table_id_addresses} using(address)
            
            """.format(table_id_traces=table_ids["traces"], table_id_addresses=table_ids["addresses"])

table_ids[current_view_name] = "{}_{}_{}_{}_{}_view".format(current_view_name, ADDRESS_SELECTION(address_selection).name, number_of_addresses, observation_period[0].replace("-","_"),observation_period[1].replace("-","_"))
view_ref = client.dataset(dataset_id).table(table_ids[current_view_name])
view = bigquery.Table(view_ref)
view.view_query = sql
view = client.create_table(view)  # API request
print("Successfully created view: '{}'".format(table_ids[current_view_name]))

Successfully created view: 'stddev_sent_tx_RICHEST_100_2020_02_01_2020_02_01_view'


Create view "active_months"

In [210]:
table_names.append("active_months")
current_view_name = table_names[-1]
sql = """
            with tx_received as (
            
                select 
                    TIMESTAMP_TRUNC(block_timestamp, MONTH, 'UTC') as month, 
                    to_address,
                    count(*) as number_tx_received
                from `masterarbeit-245718.ethereum_us.{table_id_traces}`
                group by TIMESTAMP_TRUNC(block_timestamp, MONTH, 'UTC'), to_address
                
            ), tx_sent as (
            
                select 
                    TIMESTAMP_TRUNC(block_timestamp, MONTH, 'UTC') as month, 
                    from_address,
                    count(*) as number_tx_sent
                from `masterarbeit-245718.ethereum_us.{table_id_traces}`
                group by TIMESTAMP_TRUNC(block_timestamp, MONTH, 'UTC'), from_address
                
            ), monthly_tx as (
            
                select 
                    CASE  
                      WHEN tx_sent.from_address IS NOT NULL THEN tx_sent.from_address
                      WHEN tx_received.to_address IS NOT NULL THEN tx_received.to_address
                    END AS address,
                    CASE  
                      WHEN tx_sent.month IS NOT NULL THEN tx_sent.month
                      WHEN tx_received.month IS NOT NULL THEN tx_received.month
                    END AS month,
                    ifnull(number_tx_sent,0) as number_tx_sent, 
                    ifnull(number_tx_received,0) as number_tx_received 
                from tx_sent full join tx_received 
                    on (tx_sent.from_address = tx_received.to_address and tx_sent.month = tx_received.month)
            
            ), active_months_view as (
            
                select 
                    address, 
                    countif(number_tx_sent > 0 or number_tx_received > 0) as active_months 
                from monthly_tx group by address order by address ASC 
            )
            
            select 
                address, 
                ifnull(active_months,0) as active_months 
            from `masterarbeit-245718.ethereum_us.{table_id_addresses}` left join active_months_view using(address)    
            
            """.format(table_id_addresses=table_ids["addresses"],table_id_traces=table_ids["traces"], table_id_tx=table_ids["tx"])

table_ids[current_view_name] = "{}_{}_{}_{}_{}_view".format(current_view_name, ADDRESS_SELECTION(address_selection).name, number_of_addresses, observation_period[0].replace("-","_"),observation_period[1].replace("-","_"))
view_ref = client.dataset(dataset_id).table(table_ids[current_view_name])
view = bigquery.Table(view_ref)
view.view_query = sql
view = client.create_table(view)  # API request
print("Successfully created view: '{}'".format(table_ids[current_view_name]))

Successfully created view: 'active_months_RICHEST_100_2020_02_01_2020_02_01_view'


Create table "traces_usd"

In [211]:
from urllib.request import urlopen, Request
import csv 

headers = {"User-Agent": "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3"}
url = 'https://etherscan.io/chart/etherprice?output=csv'
req = Request(url=url, headers=headers) 
html = urlopen(req).read().decode('utf-8')

with open('../data/test.csv', 'w') as file:
        file.write(html)
        
eth_usd_res = pd.read_csv('../data/test.csv')  
eth_usd = eth_usd_res.copy()
eth_usd["Date(UTC)"] = [datetime.strptime(ts, "%m/%d/%Y") for ts in eth_usd_res["Date(UTC)"]]
eth_usd = eth_usd.drop(columns=["UnixTimeStamp"])
eth_usd.columns = ["usd_eth_timestamp", "usd_eth"]
eth_usd["usd_eth"] = pd.to_numeric(eth_usd["usd_eth"])
eth_usd.to_gbq('ethereum_us.usd_eth_table', if_exists="replace")

1it [00:07,  7.58s/it]


In [212]:
table_names.append("traces_usd")
current_view_name = table_names[-1]
sql = """
             select 
                 from_address, 
                 to_address, 
                 value, 
                 status, 
                 call_type, 
                 trace_type, 
                 block_timestamp, 
                 usd_eth * value as value_usd 
             from `masterarbeit-245718.ethereum_us.{table_id_traces}` as traces_view
             left join `masterarbeit-245718.ethereum_us.usd_eth_table` as usd_eth_table 
                on (TIMESTAMP_TRUNC(usd_eth_table.usd_eth_timestamp, DAY, 'UTC') = TIMESTAMP_TRUNC(traces_view.block_timestamp, DAY, 'UTC'))
            """.format(table_id_traces=table_ids["traces"])

table_ids[current_view_name] = "{}_{}_{}_{}_{}_view".format(current_view_name, ADDRESS_SELECTION(address_selection).name, number_of_addresses, observation_period[0].replace("-","_"),observation_period[1].replace("-","_"))
view_ref = client.dataset(dataset_id).table(table_ids[current_view_name])
view = bigquery.Table(view_ref)
view.view_query = sql
view = client.create_table(view)  # API request
print("Successfully created view: '{}'".format(table_ids[current_view_name]))

Successfully created view: 'traces_usd_RICHEST_100_2020_02_01_2020_02_01_view'


Create table "usd"

In [213]:
table_names.append("usd")
current_view_name = table_names[-1]
sql = """
            with usdView as (

              with usd_receivedView as (
                    
                  -- debits
                  select to_address, sum(ifnull(value_usd, 0)) as usd_received
                  from masterarbeit-245718.ethereum_us.{table_id_traces_usd}
                  where to_address is not null
                  and status = 1
                  and (call_type not in ('delegatecall', 'callcode', 'staticcall') or call_type is null) 
                  group by to_address
                    
              ), usd_sentView as (
              
                  -- credits
                  select from_address, sum(ifnull(value_usd, 0)) as usd_sent
                  from  masterarbeit-245718.ethereum_us.{table_id_traces_usd} 
                  where from_address is not null
                  and status = 1
                  and (call_type not in ('delegatecall', 'callcode', 'staticcall') or call_type is null) 
                  group by from_address
              ) 
              select 
                  CASE 
                    when to_address is not null then to_address
                    when from_address is not null then from_address
                  end as address, 
                  ifnull(usd_received,0) as usd_received, 
                  ifnull(usd_sent,0) as usd_sent
              from usd_receivedView full outer join usd_sentView on from_address = to_address
            ) 
            select address, usd_received, usd_sent from usdView right join masterarbeit-245718.ethereum_us.{table_id_addresses} using(address)
            """.format(table_id_traces_usd=table_ids["traces_usd"], table_id_addresses=table_ids["addresses"])

table_ids[current_view_name] = "{}_{}_{}_{}_{}_view".format(current_view_name, ADDRESS_SELECTION(address_selection).name, number_of_addresses, observation_period[0].replace("-","_"),observation_period[1].replace("-","_"))
view_ref = client.dataset(dataset_id).table(table_ids[current_view_name])
view = bigquery.Table(view_ref)
view.view_query = sql
view = client.create_table(view)  # API request
print("Successfully created view: '{}'".format(table_ids[current_view_name]))

Successfully created view: 'usd_RICHEST_100_2020_02_01_2020_02_01_view'


Create View "wei_avg"

In [214]:
table_names.append("wei_avg")
current_view_name = table_names[-1]
sql = """
            with features as (
                select * from `masterarbeit-245718.ethereum_us.{table_id_wei}` 
                inner join `masterarbeit-245718.ethereum_us.{table_id_tx}` using(address) 
            )
            select 
                address,
                CASE 
                    when number_of_tx_sent > 0 THEN wei_sent / number_of_tx_sent 
                    when number_of_tx_sent = 0 THEN 0 
                END as avg_wei_sent,
                CASE 
                    when number_of_tx_received > 0 THEN wei_received / number_of_tx_received 
                    when number_of_tx_received = 0 THEN 0 
                END as avg_wei_received
            from features        
            """.format(table_id_wei=table_ids["wei"], table_id_tx=table_ids["tx"])

table_ids[current_view_name] = "{}_{}_{}_{}_{}_view".format(current_view_name, ADDRESS_SELECTION(address_selection).name, number_of_addresses, observation_period[0].replace("-","_"),observation_period[1].replace("-","_"))
view_ref = client.dataset(dataset_id).table(table_ids[current_view_name])
view = bigquery.Table(view_ref)
view.view_query = sql
view = client.create_table(view)  # API request
print("Successfully created view: '{}'".format(table_ids[current_view_name]))

Successfully created view: 'wei_avg_RICHEST_100_2020_02_01_2020_02_01_view'


Create View "usd_avg"

In [215]:
table_names.append("usd_avg")
current_view_name = table_names[-1]
sql = """
            with features as (
                select * from `masterarbeit-245718.ethereum_us.{table_id_usd}` 
                inner join `masterarbeit-245718.ethereum_us.{table_id_tx}` using(address) 
            )
            select 
                address, 
                CASE 
                    when number_of_tx_sent > 0 THEN usd_sent / number_of_tx_sent 
                    when number_of_tx_sent = 0 THEN 0 
                END as avg_usd_sent,
                CASE 
                    when number_of_tx_received > 0 THEN usd_received / number_of_tx_received 
                    when number_of_tx_received = 0 THEN 0 
                END as avg_usd_received
            from features        
            """.format(table_id_usd=table_ids["usd"], table_id_tx=table_ids["tx"])

table_ids[current_view_name] = "{}_{}_{}_{}_{}_view".format(current_view_name, ADDRESS_SELECTION(address_selection).name, number_of_addresses, observation_period[0].replace("-","_"),observation_period[1].replace("-","_"))
view_ref = client.dataset(dataset_id).table(table_ids[current_view_name])
view = bigquery.Table(view_ref)
view.view_query = sql
view = client.create_table(view)  # API request
print("Successfully created view: '{}'".format(table_ids[current_view_name]))

Successfully created view: 'usd_avg_RICHEST_100_2020_02_01_2020_02_01_view'


Create View "wei_avg_monthly"

In [216]:
table_names.append("wei_avg_monthly")
current_view_name = table_names[-1]
sql = """
            with features as (
                select * from `masterarbeit-245718.ethereum_us.{table_id_wei}` 
                inner join `masterarbeit-245718.ethereum_us.{table_id_active_months}` using(address) 
            )
            select address,
                CASE 
                    when active_months > 0 THEN wei_sent / active_months 
                    when active_months = 0 THEN 0 
                END as monthly_wei_sent,
                CASE 
                     when active_months > 0 THEN wei_received / active_months 
                     when active_months = 0 THEN 0 
                END as monthly_wei_received
                from features
            """.format(table_id_wei=table_ids["wei"], table_id_active_months=table_ids["active_months"])

table_ids[current_view_name] = "{}_{}_{}_{}_{}_view".format(current_view_name, ADDRESS_SELECTION(address_selection).name, number_of_addresses, observation_period[0].replace("-","_"),observation_period[1].replace("-","_"))
view_ref = client.dataset(dataset_id).table(table_ids[current_view_name])
view = bigquery.Table(view_ref)
view.view_query = sql
view = client.create_table(view)  # API request
print("Successfully created view: '{}'".format(table_ids[current_view_name]))

Successfully created view: 'wei_avg_monthly_RICHEST_100_2020_02_01_2020_02_01_view'


Create View "usd_avg_monthly"

In [217]:
table_names.append("usd_avg_monthly")
current_view_name = table_names[-1]
sql = """
            with features as (
                select * from `masterarbeit-245718.ethereum_us.{table_id_usd}` 
                inner join `masterarbeit-245718.ethereum_us.{table_id_active_months}` using(address) 
            )
            select address,
                CASE 
                    when active_months > 0 THEN usd_sent / active_months 
                    when active_months = 0 THEN 0 
                END as monthly_usd_sent,
                CASE 
                     when active_months > 0 THEN usd_received / active_months 
                     when active_months = 0 THEN 0 
                END as monthly_usd_received
                from features
            """.format(table_id_usd=table_ids["usd"], table_id_active_months=table_ids["active_months"])

table_ids[current_view_name] = "{}_{}_{}_{}_{}_view".format(current_view_name, ADDRESS_SELECTION(address_selection).name, number_of_addresses, observation_period[0].replace("-","_"),observation_period[1].replace("-","_"))
view_ref = client.dataset(dataset_id).table(table_ids[current_view_name])
view = bigquery.Table(view_ref)
view.view_query = sql
view = client.create_table(view)  # API request
print("Successfully created view: '{}'".format(table_ids[current_view_name]))

Successfully created view: 'usd_avg_monthly_RICHEST_100_2020_02_01_2020_02_01_view'


Create View "tx_avg_monthly"

In [218]:
table_names.append("tx_avg_monthly")
current_view_name = table_names[-1]
sql = """
            with features as (
                select * from `masterarbeit-245718.ethereum_us.{table_id_tx}` 
                inner join `masterarbeit-245718.ethereum_us.{table_id_active_months}` using(address) 
            )
            select address,
                CASE 
                    when active_months > 0 THEN number_of_tx_sent / active_months 
                    when active_months = 0 THEN 0 
                END as monthly_outgoing_txns,
                CASE 
                    when active_months > 0 THEN number_of_tx_received / active_months 
                    when active_months = 0 THEN 0 
                END as monthly_incoming_txns
            from features
            """.format(table_id_tx=table_ids["tx"], table_id_active_months=table_ids["active_months"])

table_ids[current_view_name] = "{}_{}_{}_{}_{}_view".format(current_view_name, ADDRESS_SELECTION(address_selection).name, number_of_addresses, observation_period[0].replace("-","_"),observation_period[1].replace("-","_"))
view_ref = client.dataset(dataset_id).table(table_ids[current_view_name])
view = bigquery.Table(view_ref)
view.view_query = sql
view = client.create_table(view)  # API request
print("Successfully created view: '{}'".format(table_ids[current_view_name]))

Successfully created view: 'tx_avg_monthly_RICHEST_100_2020_02_01_2020_02_01_view'


Create View "contracts_created"

In [219]:
table_names.append("contracts_created")
current_view_name = table_names[-1]

sql = """
            with contract_created_view as (
            select 
                from_address as address, 
                count(*) as number_of_contracts_created
            from `masterarbeit-245718.ethereum_us.{table_id_traces}` 
                where
                    trace_type = "create"
            group by from_address
            order by number_of_contracts_created DESC
            )
            select 
                address, 
                ifnull(number_of_contracts_created,0) as number_of_contracts_created 
            from contract_created_view right join `masterarbeit-245718.ethereum_us.{table_id_addresses}` using(address)
            """.format(table_id_traces=table_ids["traces"], table_id_addresses = table_ids["addresses"])

table_ids[current_view_name] = "{}_{}_{}_{}_{}_view".format(current_view_name, ADDRESS_SELECTION(address_selection).name, number_of_addresses, observation_period[0].replace("-","_"),observation_period[1].replace("-","_"))
view_ref = client.dataset(dataset_id).table(table_ids[current_view_name])
view = bigquery.Table(view_ref)
view.view_query = sql
view = client.create_table(view)  # API request
print("Successfully created view: '{}'".format(table_ids[current_view_name]))

Successfully created view: 'contracts_created_RICHEST_100_2020_02_01_2020_02_01_view'


Create View "contract_tx"

In [220]:
table_names.append("contract_tx")
current_view_name = table_names[-1]
sql = """
            with contract_tx_view as (
            select 
                from_address as address, 
                count(*) as contract_tx
            from `masterarbeit-245718.ethereum_us.{table_id_traces}` 
                where
                    (input != "None" and input != "0x" and trace_type = "call") or 
                    trace_type = "create" or
                    trace_type = "suicide"
            group by from_address
            order by from_address
            )
            select 
                address, 
                ifnull(contract_tx,0) as contract_tx 
            from contract_tx_view right join `masterarbeit-245718.ethereum_us.{table_id_addresses}` using(address)
            """.format(table_id_traces=table_ids["traces"], table_id_addresses = table_ids["addresses"])

table_ids[current_view_name] = "{}_{}_{}_{}_{}_view".format(current_view_name, ADDRESS_SELECTION(address_selection).name, number_of_addresses, observation_period[0].replace("-","_"),observation_period[1].replace("-","_"))
view_ref = client.dataset(dataset_id).table(table_ids[current_view_name])
view = bigquery.Table(view_ref)
view.view_query = sql
view = client.create_table(view)  # API request
print("Successfully created view: '{}'".format(table_ids[current_view_name]))

Successfully created view: 'contract_tx_RICHEST_100_2020_02_01_2020_02_01_view'


Create View "diff_token_used"

In [221]:
table_names.append("diff_token_used")
current_view_name = table_names[-1]
sql = """
            with token_transfers as (
                select * from bigquery-public-data.crypto_ethereum.token_transfers 
                where DATE(block_timestamp) >= '{start}' and DATE(block_timestamp) <= '{end}'
            ) 
            select 
                address, 
                count(DISTINCT token_address) as diff_token_used
            from token_transfers right join `masterarbeit-245718.ethereum_us.{table_id_addresses}` on from_address = address 
            group by address
            order by diff_token_used DESC
            """.format(start=observation_period[0], end=observation_period[1], table_id_addresses=table_ids["addresses"]); 

table_ids[current_view_name] = "{}_{}_{}_{}_{}_view".format(current_view_name, ADDRESS_SELECTION(address_selection).name, number_of_addresses, observation_period[0].replace("-","_"),observation_period[1].replace("-","_"))
view_ref = client.dataset(dataset_id).table(table_ids[current_view_name])
view = bigquery.Table(view_ref)
view.view_query = sql
view = client.create_table(view)  # API request
print("Successfully created view: '{}'".format(table_ids[current_view_name]))

Successfully created view: 'diff_token_used_RICHEST_100_2020_02_01_2020_02_01_view'


## Merge Views

In [222]:
# Remove non-feature views
for name in ["traces","addresses", "traces_usd", "transactions", "blocks"]:
    if name in table_names:
        table_names.remove(name)

In [223]:
current_view_name = "features"
sql = "select * from "

for index, view_name in enumerate(table_names):
    sql += "masterarbeit-245718.ethereum_us.{}".format(table_ids[view_name])
    if index != 0:
        sql += " using(address)"
    if index != (len(table_names) - 1):
        sql += " inner join "

table_ids[current_view_name] = "{}_{}_{}_{}_{}_view".format(current_view_name, ADDRESS_SELECTION(address_selection).name, number_of_addresses, observation_period[0].replace("-","_"),observation_period[1].replace("-","_"))
view_ref = client.dataset(dataset_id).table(table_ids[current_view_name])
view = bigquery.Table(view_ref)
view.view_query = sql
view = client.create_table(view)  # API request
print("Successfully created view: '{}'".format(table_ids[current_view_name]))

Successfully created view: 'features_RICHEST_100_2020_02_01_2020_02_01_view'


## Estimate Costs

In [224]:
table_name = "features"
table_id = "{}_{}_{}_{}_{}".format(table_name, ADDRESS_SELECTION(address_selection).name, number_of_addresses, observation_period[0].replace("-","_"),observation_period[1].replace("-","_"))

job_config = bigquery.QueryJobConfig(destination="masterarbeit-245718.ethereum_us.{table_id}".format(table_id = table_id))
job_config.dry_run = True
job_config.use_query_cache = False

sql = """
    SELECT *
    FROM `masterarbeit-245718.ethereum_us.{table_id_features}` 
""".format(table_id_features = table_ids["features"])

# Start the query, passing in the extra configuration.
query_job = client.query(sql, job_config=job_config)  # Make an API request.

cost_dollars = (query_job.total_bytes_processed / 10 ** 12) * 5

print("{} Megabytes will be processed".format(round(query_job.total_bytes_processed / 10 ** 6)))
print("It will cost ${}.".format(cost_dollars))

587 Megabytes will be processed
It will cost $0.00293490337.


## Execute Query (WARNING: this operation costs money!)

In [225]:
table_name = "features"
table_id = "{}_{}_{}_{}_{}".format(table_name, ADDRESS_SELECTION(address_selection).name, number_of_addresses, observation_period[0].replace("-","_"),observation_period[1].replace("-","_"))

job_config = bigquery.QueryJobConfig(destination="masterarbeit-245718.ethereum_us.{table_id}".format(table_id = table_id))

sql = """
    SELECT *
    FROM `masterarbeit-245718.ethereum_us.{table_id_features}` 
""".format(table_id_features = table_ids["features"])

# Start the query, passing in the extra configuration.
if (cost_dollars > max_bigquery_costs_usd):
    raise Exception("Warning: This operation costs $ {}.".format(cost_dollars))

query_job = client.query(sql, job_config=job_config)  # Make an API request.
result = query_job.result()

print("Successfully created table: '{}'".format(table_id))

Successfully created table: 'features_RICHEST_100_2020_02_01_2020_02_01'


# Temporary Scripts

In [226]:
# data = result.to_dataframe(); 

# display(data)

# if (data.isna().values.any() == True):
#     raise Exception("Null values deteceted.")

In [227]:
# table_id = "{}_{}_{}_{}_{}_tmp".format(current_view_name, ADDRESS_SELECTION(address_selection).name, number_of_addresses, observation_period[0].replace("-","_"),observation_period[1].replace("-","_"))

# job_config = bigquery.QueryJobConfig(destination="masterarbeit-245718.ethereum_us.{table_id}".format(table_id = table_id))

# sql = """
#     SELECT *
#     FROM `masterarbeit-245718.ethereum_us.{table_id_diff_token_used}` 
# """.format(table_id_diff_token_used = table_ids["diff_token_used"])

# # Start the query, passing in the extra configuration.
# query_job = client.query(sql, job_config=job_config)  # Make an API request.
# data = query_job.result().to_dataframe(); 

# data